### **title 기반 Semantic Alignment(의미적 일치성) 설계**

**Semantic Alignment (의미적 일치성 - Vibe Check)**

- **Goal:** 사용자의 의도(Taxonomy)와 추천된 '노래의 분위기/가사'가 일치하는가?
- **Primary KPI:** LLM Judge에게 "이 상황(독서실)에 이 노래(Bang Bang)가 어울리는가?"를 묻고 **평균 4.5점** 달성.
- **Guardrail KPI:** 상황과 정반대되는 노래(예: 수면에 락, 이별 위로에 신나는 댄스) 추천 발생률 **0% 유지**.

평가 로직: LLM Judge가 "추천된 노래 제목"과 "GPT의 추천 사유(Reasoning)"를 보고, LLM의 지식을 기반으로 이 상황에 맞는지 5점 만점으로 채점합니다.

Primary KPI: LLM Judge 평균 4.5점 달성

Guardrail KPI: 치명적 오류율, 상황과 정반대되는 노래 추천 발생율 0% 유지

In [ ]:
import pandas as pd
from openai import OpenAI
import json
import time

# ===========================================================
# 1. 설정 (Configuration)
# ===========================================================
API_KEY = ""
client = OpenAI(api_key=API_KEY)

INPUT_FILE = "evaluation_set_v1.csv"            # 아까 만든 평가셋
OUTPUT_FILE = "semantic_result.xlsx" # 최종 결과 파일

# ===========================================================
# 2. 프롬프트 설계 (최종 확정 스키마 반영)
# ===========================================================

# [DJ 역할] 백엔드에 전달한 최종 스키마 그대로 적용
SYSTEM_PROMPT_DJ = """
# Role
당신은 사용자의 상황(Context)과 취향(Preference)에 맞춰, Spotify에 실존하는 음악을 추천하는 AI DJ입니다.

# Critical Constraints
1. **Real Song Only**: 반드시 실제로 존재하는 곡을 추천해야 합니다.
2. **Context Alignment**: 목표(Goal)와 소음(Decibel)을 최우선으로 고려하세요.

#  Primary Tag Generation Rules (Strict)
To facilitate data analysis, you must generate the 'primary_tag' following this strict format:

1. **Format**: `"{Goal}_{Genre}"` (Snake case)
2. **Prefix (Goal)**: Use the exact 'Goal' from the user input.
   - Options: focus, relax, sleep, active, anger, consolation, neutral
3. **Suffix (Genre)**: Choose the most representative genre from this list ONLY.
   - Options: pop, k-pop, rock, hip-hop, r-nb, jazz, classical, electronic, lo-fi, ambient, soundtrack, acoustic, metal, indie, folk
4. **Example**: `focus_classical`, `active_rock`, `sleep_ambient`

# Output JSON Structure
{
  "recommendation_meta": {
    "reasoning": "추천 이유 (한글 1문장)",
    "primary_tag": "Goal_Genre format (e.g., focus_lofi)"
  },
  "track_info": {
    "artist_name": "Exact Artist Name",
    "track_id": "Exact Song Title Spotify ID",
    "track_title": "Exact Song Title"
  },
  "target_audio_features": { 
    "min_tempo": int,
    "max_tempo": int,
    "target_energy": (float 0.0~1.0),
    "target_instrumentalness": (float 0.0~1.0)
    "target_valence": (float 0.0~1.0),
    "target_acousticness": (float 0.0~1.0)
  }
} 
"""

# [Judge 역할] 노래 제목과 이유를 보고 적합성 평가
SYSTEM_PROMPT_JUDGE = """
# Role
당신은 음악 평론가이자 QA Judge입니다.
AI가 추천한 '노래(Song)'와 '추천 사유(Reasoning)'가 주어진 '상황(Context)'에 적합한지 평가합니다.

# Task
1. 추천된 노래(Artist - Title)가 실제로 존재하는지, 어떤 분위기인지 당신의 지식(Knowledge Base)을 통해 파악하세요.
2. 추천 사유(Reasoning)가 상황에 맞게 논리적인지 확인하세요.

# Scoring Criteria (1-5)
- 5점 (Perfect): 상황에 완벽하게 어울리는 명곡이며, 추천 사유도 논리적임.
- 3점 (Fair): 나쁘지 않으나, 베스트 초이스는 아님.
- 1점 (Fail): 상황과 정반대(분위기 파괴) 또는 존재하지 않는 노래(Hallucination).

# Output JSON
{"score": int, "reason": "string"}
"""

# ===========================================================
# 3. 함수 정의
# ===========================================================

def get_song_recommendation(row):
    """ [Step 1] GPT에게 최종 스키마로 추천 요청 """
    user_msg = f"""
    Location: {row['Location']}
    Decibel: {row['Decibel']}
    Goal: {row['Goal']}
    Preference: {row['User Pref (Genre/Artist)']}
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_DJ},
                {"role": "user", "content": user_msg}
            ],
            response_format={"type": "json_object"},
            temperature=0.7
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f" Gen Error: {e}")
        return {}

def evaluate_semantic_alignment(row, artist, title, reasoning, target_features):
    """ [Step 2] LLM Judge에게 의미적 일치성 평가 요청 """
    judge_msg = f"""
    [User Context]
    Location: {row['Location']}, Goal: {row['Goal']}, Decibel: {row['Decibel']}
    User Pref: {row['User Pref (Genre/Artist)']}
    
    [AI Recommendation]
    Song: {artist} - {title}
    Reasoning: {reasoning}
    Target Features (AI Intention): {target_features}
    
    [Expected Vibe]
    {row['Expected Keywords / Vibe (기대 결과)']}
    
    위 노래와 추천 사유가 이 상황에 적합한지 평가해주세요.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_JUDGE},
                {"role": "user", "content": judge_msg}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        return {"score": 0, "reason": f"Error: {e}"}

# ===========================================================
# 4. 메인 실행 (수정됨)
# ===========================================================
print("\n [Final Schema] 의미적 일치성 평가 시작...")
df = pd.read_csv(INPUT_FILE)
results = []

for index, row in df.iterrows():
    print(f"▶ Case {index + 1}: {row['Location']} ({row['Goal']})...", end=" ")
    
    # 1. 추천 생성
    gpt_output = get_song_recommendation(row)
    
    # 데이터 파싱
    meta = gpt_output.get('recommendation_meta', {})
    track = gpt_output.get('track_info', {})
    target = gpt_output.get('target_audio_features', {})
    
    artist = track.get('artist_name', "Unknown")
    title = track.get('track_title', "Unknown")
    reasoning = meta.get('reasoning', "No reasoning")
    # ▼ [추가됨] GPT가 설정한 분위기 태그 가져오기
    primary_tag = meta.get('primary_tag', "Unknown") 
    
    print(f" {artist} - {title} (Tag: {primary_tag})")
    
    # 2. 품질 평가 (LLM Judge)
    judge_result = evaluate_semantic_alignment(row, artist, title, reasoning, target)
    
    # 3. 저장
    results.append({
        "Case ID": row['ID'],
        "Scenario": f"{row['Location']} / {row['Goal']}",
        "Conflict Type": row['Type'],
        
        # 추천 결과
        "Recommended Artist": artist,
        "Recommended Title": title,
        "Primary Tag": primary_tag, #  엑셀에 저장!
        "Reasoning": reasoning,
        
        # [수정됨] 분석용 Target Features 전체 저장 (GPT의 의도)
        "Target Energy": target.get('target_energy', None),
        "Target Valence": target.get('target_valence', None),
        "Target Instrumentalness": target.get('target_instrumentalness', None),
        "Target Acousticness": target.get('target_acousticness', None),
        "Target Min Tempo": target.get('min_tempo', None),
        "Target Max Tempo": target.get('max_tempo', None),

        # 평가 점수
        "LLM Score": judge_result['score'],
        "Judge Reason": judge_result['reason']
    })
    
    time.sleep(0.5)


# ===========================================================
# 5. 결과 저장 및 KPI 확인
# ===========================================================
result_df = pd.DataFrame(results)
result_df.to_excel(OUTPUT_FILE, index=False)

avg_score = result_df['LLM Score'].mean()
fatal_errors = len(result_df[result_df['LLM Score'] == 1])

print("\n" + "="*50)
print(f" 평가 완료! 파일: {OUTPUT_FILE}")
print(f" [KPI 1] 의미적 일치성 평균 점수: {avg_score:.2f} / 5.0 (목표: 4.5)")
print(f" [KPI 2] 치명적 오류(1점) 발생: {fatal_errors}건 (목표: 0건)")
print("="*50)


 [Final Schema] 의미적 일치성 평가 시작...
▶ Case 1: library (focus)...  Ludovico Einaudi - Nuvole Bianche (Tag: focus_classical)
▶ Case 2: gym (active)...  AC/DC - Thunderstruck (Tag: active_rock)
▶ Case 3: home (sleep)...  Max Richter - Dream 3 (In the Midst of My Life) (Tag: sleep_ambient)
▶ Case 4: cafe (relax)...  Miles Davis - So What (Tag: relax_jazz)
▶ Case 5: park (active)...  NewJeans - Hype Boy (Tag: active_k-pop)
▶ Case 6: co-working (focus)...  Lo-fi Beats - Chill Study Beats (Tag: focus_lofi)
▶ Case 7: moving (neutral)...  Dua Lipa - Levitating (Tag: neutral_pop)
▶ Case 8: home (consolation)...  IU - Through the Night (Tag: consolation_k-pop)
▶ Case 9: library (focus)...  Metallica - Nothing Else Matters (Tag: focus_metal)
▶ Case 10: gym (active)...  Ludwig van Beethoven - Symphony No. 5 in C Minor, Op. 67: I. Allegro con brio (Tag: active_classical)
▶ Case 11: home (sleep)...  J. Cole - Apparently (Tag: sleep_hip-hop)
▶ Case 12: co-working (focus)...  BTS - Euphoria (Tag: focus_k

---

GPT가 키워드 생성이 아닌 제목을 생성하기 떄문에 임베딩 못함(임베딩 모델은 밤편지와 독서실의 거리 파악을 못함) 정성적 평가에 의존해야함

-> 정성적 평가의 리스크 존재: GPT-4o가 세상의 모든 노래를 알진 못함
1. 지식의 한계, GPT가 모르는 최신곡이나 인디 음악이 나오면 "모르겠지만 제목만 보니 어울릴 것 같네요"라고 대충 채점할 수도 있음
2. 주관적 편향, 힙합은 시끄러우니까 독서실에 안 돼"라고 편견을 가질 수 있음. (Lofi Hip-hop은 되는데)

-> 단일 지표에 의존하지 않고 '하이브리드 검증(Hybrid Validation)' 시스템을 구축
1. LLM Judge를 통해 노래의 '문화적 맥락과 분위기'를 1차로 평가하고,
2. 실제 Spotify API에서 추출한 '오디오 데이터(BPM, Energy)'로 2차 '객관적 수치 검증'을 수행.


오류가 발생해서 오류로그가 뜨면 프롬프트에 해당 오류 관련 규칙 추가
오류 없으면 프롬프트 배포 확정